# Projeto 1 - Ciência dos Dados

Nome: Maria Carolina Porto 

Nome: Thomas Kassabian

Nome: Kaique Tinto

Nome: Eduardo Candeias

___
Carregando algumas bibliotecas:

In [56]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [57]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\carol\OneDrive - Insper - Institudo de Ensino e Pesquisa\2 SEMESTRE\C_DADOS\Projetos\CDADOS_Projeto1


Carregando a base de dados com as mensagens dos seus arquivos:

In [58]:
train = pd.read_csv('dados_treino_QUARTETO_Thomas.csv')
train

,Review,NPS
0,Bangkok to Phuket round trip. The lounge at th...,Promoter
1,A real mixed bag with Air New Zealand from Auc...,Passive
2,Second in the queue in business class check-...,Detractor
3,London Heathrow to Riyadh return. Pleasant f...,Promoter
4,Hong Kong to Bangkok. Check-in at the transf...,Promoter
...,...,...
4195,Ft Lauderdale to Kansas City via Detroit. I ...,Promoter
4196,After 3 hours in the plane waiting for repairs...,Detractor
4197,ARN-LHR on an ageing but generally clean B737-...,Detractor
4198,My mother left on AI 43 MAA-DEL connecting AI ...,Promoter


In [59]:
test = pd.read_csv('dados_teste_QUARTETO_Thomas.csv')
test

,Review,NPS
0,Melbourne to Singapore on a Singapore Airlines...,Promoter
1,Palermo to Moscow via Rome and I was so unluc...,Detractor
2,Actual flight both ways was good but the handl...,Passive
3,Flew Spirit Airlines from Orlando to Boston....,Detractor
4,"Shockingly poor experience on many levels, as...",Detractor
...,...,...
1795,I broke my foot on my vacation and the entire ...,Promoter
1796,LHR-PEK-Tokyo RT in Business. One-word comment...,Passive
1797,Gatwick to Fort Lauderdale. Charging to choos...,Detractor
1798,"Flown with Aeroflot for the first time, from L...",Passive


___
## Classificador automático


Esse classificador tem por objetivo ensinar um Boot a classificar corretamente os reviews de clientes de uma companhia aérea. Para isso, utilizamos um dataFrame "Train" para treiná-lo, e um dataFrame distinto "Test" para testá-lo.

As possíveis classificações são:
* Detractor: Clientes que expressaram insatisfação ou descontentamento significativo com a companhia aérea;
* Promoter: Clientes extremamente satisfeitos e leais à companhia aérea;
* Passive: Clientes que não expressaram satisfação ou insatisfação com a companhia aérea.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [60]:
# Função que limpa um texto

import re

def cleanup(text):    
    # Retira a pontuação
    punctuation = '[´"!-.:?;$''()]'
    pattern = re.compile(punctuation)
    
    # Retira as stopwords (palavras indiferentes para o cálculo da probabilidade)
    stopwords = ['the','to', 'and', 'was', 'i', 'a', 'in', 'on', 'of', 'with','for', 'flight']
    words = r'\b(?:' + '|'.join(map(re.escape, stopwords)) + r')\b'
    
    # Retira palavras de duas letras
    limpa_duas_letras = r'\b\w{1,2}\b'
    

    # Aplica as alterações e retorna o texto limpo
    text = re.sub(pattern, '', text)
    text = re.sub(words, '', text, flags=re.IGNORECASE)
    text = re.sub(limpa_duas_letras, '', text)

    return text

In [61]:
# Função que recebe um dataframe, limpa e transforma em lista
def transf_lista(dataframe):
    texto = " "

    for linha in dataframe['Review']:
        texto += cleanup(linha)
    
    lista = texto.lower().split()
    return lista

In [62]:
train.NPS.value_counts()

Detractor    1965
Promoter     1549
Passive       686
Name: NPS, dtype: int64

In [69]:
# Criando pd.Series para cada classificação distinta e para o total
# pdSeires com repetição de palavras

serie_train = pd.Series(transf_lista(train))
serie_Detractor = pd.Series(transf_lista(train.loc[(train.NPS == 'Detractor'), :]))
serie_Promoter = pd.Series(transf_lista(train.loc[(train.NPS == 'Promoter'), :]))
serie_Passive = pd.Series(transf_lista(train.loc[(train.NPS == 'Passive'), :]))

In [70]:
# Criando uma tabela com a contagem de cada palavra em cada classificação
# pdSeries sem repetição de palavras

tabela_train = serie_train.value_counts()
tabela_Detractor = serie_Detractor.value_counts()
tabela_Promoter = serie_Promoter.value_counts()
tabela_Passive = serie_Passive.value_counts()

In [72]:
# Criando uma tabela com a porcentagem relativa de cada palavra em cada classificação

tabela_Detractor_relativo = serie_Detractor.value_counts(True)
tabela_Promoter_relativo = serie_Promoter.value_counts(True)
tabela_Passive_relativo = serie_Passive.value_counts(True)

___
### Calculando as porcentagens


In [40]:
# Função que calcula a probabilidade de uma palavra estar em um NPS (classificação) específico

# Recebe: palavra -> palavra que se deseja calcular a probabilidade;
#         tab_NPS -> tabela de um NPS específico;
#         len_serie_NPS -> quantidade de palavras totais (com repetição) em um NPS específico.

def calc_prob(palavra, tab_NPS, len_serie_NPS):
    prob = 1
    # Se a palavra estiver na tabela do NPS específico
    if palavra in tab_NPS:
        prob = ((tab_NPS[palavra] + 1)/(len_serie_NPS+ len(tabela_train)))*1e3
        
    # Se a palavra for inédita
    else:   
        prob = (1/(len_serie_NPS + len(tabela_train)))*1e3

    return prob

In [74]:
# Probabilidade da palavra ser de cada NPS específico em relação ao total

P_D = len(serie_Detractor)/len(serie_train)
P_Pa = len(serie_Passive)/len(serie_train)
P_Pr = len(serie_Promoter)/len(serie_train)

In [77]:
# Classifica a frase em detractor, promoter, passive

classif = []

# For que percorre todos os reviews do dataFrame
for frase in train.Review:

    #Limpar a frase
    frase_limpa = cleanup(frase)
    frase_limpa = frase_limpa.lower().split()
    
    P_frase_dado_D = 1
    P_frase_dado_Pa = 1 
    P_frase_dado_Pr = 1

    # For que percorre cada palavra das frases
    for palavra in frase_limpa:

        #Probabilidade da frase ser de cada NPS específico
        P_frase_dado_D *= calc_prob(palavra, tabela_Detractor, len(serie_Detractor))
        P_frase_dado_Pa *= calc_prob(palavra, tabela_Passive, len(serie_Passive))
        P_frase_dado_Pr *= calc_prob(palavra, tabela_Promoter, len(serie_Promoter))

    #Probabilidade de cada NPS específico para a frase
    P_D_dado_frase = (P_frase_dado_D * P_D)
    P_Pa_dado_frase = (P_frase_dado_Pa * P_Pa)
    P_Pr_dado_frase = (P_frase_dado_Pr * P_Pr)

    # Armazena as probabilidades, compara e adiciona a maior classif na lista 
    prob = [P_D_dado_frase, P_Pa_dado_frase, P_Pr_dado_frase]

    if max(prob) == prob[0]:
        classif.append("Detractor")
    elif max(prob) == prob[1]:
        classif.append("Passive")
    elif max(prob) == prob[2]:
        classif.append("Promoter")
        
# Cria uma nova coluna com as classificações do Boot
train['Boot'] = classif
train


,Review,NPS,Boot
0,Bangkok to Phuket round trip. The lounge at th...,Promoter,Promoter
1,A real mixed bag with Air New Zealand from Auc...,Passive,Passive
2,Second in the queue in business class check-...,Detractor,Detractor
3,London Heathrow to Riyadh return. Pleasant f...,Promoter,Promoter
4,Hong Kong to Bangkok. Check-in at the transf...,Promoter,Promoter
...,...,...,...
4195,Ft Lauderdale to Kansas City via Detroit. I ...,Promoter,Detractor
4196,After 3 hours in the plane waiting for repairs...,Detractor,Detractor
4197,ARN-LHR on an ageing but generally clean B737-...,Detractor,Promoter
4198,My mother left on AI 43 MAA-DEL connecting AI ...,Promoter,Promoter


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [43]:
certo = train.loc[train.NPS == train.Boot, :]
certo

,Review,NPS,Boot
0,Bangkok to Phuket round trip. The lounge at th...,Promoter,Promoter
1,A real mixed bag with Air New Zealand from Auc...,Passive,Passive
2,Second in the queue in business class check-...,Detractor,Detractor
3,London Heathrow to Riyadh return. Pleasant f...,Promoter,Promoter
4,Hong Kong to Bangkok. Check-in at the transf...,Promoter,Promoter
...,...,...,...
4193,Great Christmas holiday in Spain starting wit...,Promoter,Promoter
4194,Gothenburg to London. Flights were on time. S...,Detractor,Detractor
4196,After 3 hours in the plane waiting for repairs...,Detractor,Detractor
4198,My mother left on AI 43 MAA-DEL connecting AI ...,Promoter,Promoter


In [44]:
pd.crosstab(train.NPS, train.Boot)

Boot,Detractor,Passive,Promoter
NPS,,,
Detractor,1751,110,104
Passive,109,412,165
Promoter,113,1,1435


___
### Testando com o dataFrame "Test"

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens. Ex: stemming, lemmatization, stopwords
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**